### 9. 추천 시스템
* 협업 필터링: 사용자가 아이템에 매긴 평점 정보나 상품 구매 이력과 같은 사용자 행동 양식만을 기반으로 추천을 수행하는 것
    * 최근접 이웃 방식
        * 사용자 기반 : 특정 사용자와 유사한 다른 사용자를 TOP-N으로 선정해 이 TOP-N 사용자가 좋아하는 아이템을 추천하는 방식. 특정 사용자와 타 사용자 간의 유사도를 측정한 뒤 유사도가 가장 높은 TOP-N 사용자를 추출해 그들이 선호하는 아이템을 추천하는 것
        * 아이템 기반 : 아이템을 좋아하는지/싫어하는지의 평가 척도가 유사한 아이템을 추천하는 기준이 되는 알고리즘. 
        * 아이템 기반 협업 필터링이 정확도가 더 높음
    * 잠재 요인 방식
        * 사용자-아이템 평점 매트릭스 속에 숨어있는 잠재 요인을 추출해 추천 예측을 할 수 있게 하는 기법
        * 행렬 분해
            * 주로 SVD방식을 이용하나 Null값이 있는 경우 사용하지 못함 -> 경사하강법 이용
        * 경사하강법
            * P와 Q 행렬로 계산된 예측 R행렬 값이 실제 R 행렬 값과 가장 최소의 오류를 가질 수 있도록 반복적인 비용함수 최적화를 통해 P와 Q를 유추

**경사하강법을 이용해 행렬분해를 수행하는 과정**

In [2]:
import numpy as np

# 원본 행렬 R생성, 분해 행렬 P와 Q 초기화, 잠재 요인 차원 K는 3으로 설정
R=np.array([[4, np.NaN, np.NaN, 2, np.NaN],
           [np.NaN, 5, np.NaN, 3, 1], 
           [np.NaN, np.NaN, 3, 4, 4],
           [5, 2, 1, 2, np.NaN]])

num_users, num_items=R.shape
K=3

# P와 Q행렬의 크기를 지정하고 정규 분포를 가진 임의의 값으로 입력
np.random.seed(1)
P=np.random.normal(scale=1./K, size=(num_users, K))
Q=np.random.normal(scale=1./K, size=(num_items, K))

**실제 행렬과 예측 행렬의 오차 구하기**

In [3]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error=0;
    # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 행렬 R 생성
    full_pred_matrix=np.dot(P, Q.T)
    
    # 실제 R행렬에서 널이 아닌 값의 위치 인덱스를 추출해 실제 R행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind=[non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind=[non_zero[1] for non_zero in non_zeros]
    R_non_zeros=R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros=full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse=mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse=np.sqrt(mse)
    
    return mse

In [5]:
# R>0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
non_zeros=[(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j]>0]

steps=1000 # 반복 수행 횟수
learning_rate=0.01
r_lambda=0.01 # L2규제 계수

# SGD 기법으로 P와 Q 계속 업데이트
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류값 구함
        eij=r-np.dot(P[i, :], Q[j, :].T)
        # regularization을 반영한 SGD 업데이트
        P[i, :]=P[i, :]+learning_rate*(eij*Q[j, :]-r_lambda*P[i, :])
        Q[j, :]=Q[j, :]+learning_rate*(eij*Q[i, :]-r_lambda*Q[j, :])
        
        rmse=get_rmse(R, P, Q, non_zeros)
        if(step%50)==0:
            print('### iteration step: ', step, ' rmse: ', rmse)

### iteration step:  0  rmse:  10.659037853664685
### iteration step:  0  rmse:  10.663183505301207
### iteration step:  0  rmse:  10.645613047299983
### iteration step:  0  rmse:  10.644346084939555
### iteration step:  0  rmse:  10.645581949897782
### iteration step:  0  rmse:  10.643328122261329
### iteration step:  0  rmse:  10.626349009530133
### iteration step:  0  rmse:  10.622087609323627
### iteration step:  0  rmse:  10.611045186349452
### iteration step:  0  rmse:  10.614228003433338
### iteration step:  0  rmse:  10.61433525373386
### iteration step:  0  rmse:  10.612784895036869
### iteration step:  50  rmse:  0.4826847637377499
### iteration step:  50  rmse:  0.4809214258571381
### iteration step:  50  rmse:  0.4787702501177038
### iteration step:  50  rmse:  0.47840583761081873
### iteration step:  50  rmse:  0.476797573736393
### iteration step:  50  rmse:  0.4744591769196104
### iteration step:  50  rmse:  0.46878465911555617
### iteration step:  50  rmse:  0.458334909

### iteration step:  650  rmse:  0.006871060826379036
### iteration step:  650  rmse:  0.006871161463091513
### iteration step:  650  rmse:  0.006877224254368025
### iteration step:  650  rmse:  0.006830245619597101
### iteration step:  650  rmse:  0.006840417280251347
### iteration step:  650  rmse:  0.00684872896133637
### iteration step:  650  rmse:  0.006812559002439696
### iteration step:  650  rmse:  0.006802023629205184
### iteration step:  650  rmse:  0.006743517331703443
### iteration step:  650  rmse:  0.006740242215160119
### iteration step:  650  rmse:  0.006743599006297992
### iteration step:  650  rmse:  0.006822987985846849
### iteration step:  700  rmse:  0.00614899094321202
### iteration step:  700  rmse:  0.006148079963171762
### iteration step:  700  rmse:  0.0061534294737563945
### iteration step:  700  rmse:  0.006111062455266679
### iteration step:  700  rmse:  0.0061213042029873805
### iteration step:  700  rmse:  0.0061297429351423955
### iteration step:  700  r

In [6]:
pred_matrix=np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[4.021 1.912 1.192 1.941 1.27 ]
 [4.314 4.995 1.991 2.949 1.015]
 [6.707 5.082 3.004 3.949 3.951]
 [4.91  1.97  0.985 2.157 0.188]]


널인 값은 새로운 예측값으로 채워짐